In [4]:
import sys
import numpy as np
import random
import os
import math
import warnings
warnings.filterwarnings('ignore')
import time
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib
from matplotlib import animation
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

import importlib
import scripts.theodolite_function as tfu
import scripts.theodolite_utils as ttf
import GPy
import scripts.gp_prediction_utils as GPf
tfu = importlib.reload(tfu)
ttf = importlib.reload(ttf)
GPf = importlib.reload(GPf)

from scipy.interpolate import splprep, splev
from scipy import interpolate
from scipy import spatial
import seaborn as sns

from scipy.spatial.transform import Rotation as R
from IPython.display import HTML
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, ConstantKernel)
from stheno import B, Measure, GP, EQ, Delta
import torch

from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from wbml.plot import tweak


In [7]:
# Read rosbag of grand axe and show the trajectory
ttf = importlib.reload(ttf)
tfu = importlib.reload(tfu)
GPf = importlib.reload(GPf)


# file = ["/home/norlab/Data/IROS_2022/20220224_TS/2022-02-24-15-34-38.bag",
#         "/home/norlab/Data/IROS_2022/20220307_TS/2022-03-07-19-20-06.bag",
#         "/home/norlab/Data/IROS_2022/20220312_TS/2022-03-12-09-45-12.bag",
#         "/home/norlab/Data/IROS_2022/20220314_TS/2022-03-14-10-47-49.bag",
#         "/home/norlab/Data/IROS_2022/20220316_TS/2022-03-16-19-02-42.bag",
#         "/home/norlab/Data/IROS_2022/20220331_TS/2022-03-31-10-22-52.bag",
#         "/home/norlab/Data/IROS_2022/20220331_TS/2022-03-31-11-20-05.bag",
#         "/home/norlab/Data/IROS_2022/20220427_TS/2022-04-27-12-12-10_filered.bag",
#         "/home/norlab/Data/IROS_2022/20220505_TS/empty1_2022-05-05-19-14-33.bag",
#         "/home/norlab/Data/IROS_2022/20220505_TS/cones1_2022-05-05-19-25-54.bag",
#         "/home/norlab/Data/IROS_2022/20201007_Foret/2020-10-08-11-57-30_one_prism_continuous_mapping_quarry1_filtered.bag",
#         "/home/norlab/Data/IROS_2022/20201007_Foret/2020-10-08-12-14-23_three_prism_continuos_mapping_quarry2_filtered.bag",
#         "/home/norlab/Data/IROS_2022/20201007_Foret/2020-10-08-17-58-38_ski_trail_continuous_motion_filtered.bag",
#         "/home/norlab/Data/IROS_2022/20201007_Foret/2020-10-08-18-01-19_ski_trail_start_stop_motion_filtered.bag",
#         "/home/norlab/Data/IROS_2022/20201007_Foret/2020-10-08-18-05-08_ski_trail_start_stop_straight_segment_filtered.bag",
#         "/home/norlab/Data/IROS_2022/20201201_Foret/2020-12-01-17-29-01_theodolite.bag",
#         "/home/norlab/Data/IROS_2022/20201201_Foret/2020-12-01-18-00-21_theodolite.bag",
#         "/home/norlab/Data/IROS_2022/20201201_Foret/2020-12-01-18-15-43_theodolite.bag",
#         "/home/norlab/Data/IROS_2022/20201202_Foret/2020-12-02-11-37-05_theodolite.bag",
#         "/home/norlab/Data/IROS_2022/20201202_Foret/2020-12-02-12-28-41_theodolite.bag",
#         "/home/norlab/Data/IROS_2022/20201203_Foret/2020-12-03-10-21-59_theodolite.bag",
#         "/home/norlab/Data/IROS_2022/20210222_Foret/2021-02-22-15-43-34_filtered.bag",
#         "/home/norlab/Data/IROS_2022/20210222_Foret/2021-02-22-15-50-23_filtered.bag",
#         "/home/norlab/Data/IROS_2022/20210222_Foret/2021-02-22-16-01-10_filtered.bag"]

# output = ["TS/20220224/",
#          "TS/20220307/",
#          "TS/20220312/",
#          "TS/20220314/",
#          "TS/20220316/",
#          "TS/20220331-1/",
#          "TS/20220331-2/",
#          "TS/20220427/",
#          "TS/20220505_empty/",
#          "TS/20220505_cones/",
#          "TS/20201007/01/",
#          "TS/20201007/02/",
#          "TS/20201007/03/",
#          "TS/20201007/04/",
#          "TS/20201007/05/",
#          "TS/20201201/01/",
#          "TS/20201201/02/",
#          "TS/20201201/03/",
#          "TS/20201202/01/",
#          "TS/20201202/02/",
#          "TS/20201203/01/",
#          "TS/20210222/01/",
#          "TS/20210222/02/",
#          "TS/20210222/03/"]

'''
file = ["/media/will/T7/theodolite_bags/20220715_TS/2022-07-15-11-50-15.bag",
        "/media/will/T7/theodolite_bags/20220715_TS/2022-07-15-11-53-12.bag",
        "/media/will/T7/theodolite_bags/20220715_TS/2022-07-15-11-58-58.bag",
        "/media/will/T7/theodolite_bags/20220715_TS/2022-07-15-14-42-27.bag"]
        # "/media/will/T7/theodolite_bags/20220513_TS_2/2022-05-13-22-35-30.bag",
        # "/media/will/T7/theodolite_bags/20220513_TS_2/2022-05-13-22-51-40.bag"]

output = ["TS/20220717/01/",
          "TS/20220717/02/",
          "TS/20220717/03/",
          "TS/20220717/04/",
          "TS/20220717/05/"]
          # "TS/20220513/06/"]
          
'''
        
file = ["/home/norlab/Data/IROS_2022/20220717_TS/constrained_2022-07-17-20-10-31.bag",
        "/home/norlab/Data/IROS_2022/20220717_TS/constrained_2022-07-17-20-11-10.bag",
        "/home/norlab/Data/IROS_2022/20220717_TS/cones_2022-07-17-20-38-29.bag",
        "/home/norlab/Data/IROS_2022/20220717_TS/empty_2022-07-17-21-03-14.bag",
        "/home/norlab/Data/IROS_2022/20220717_TS/calibration_2022-07-17-21-27-28.bag"
       ]

output = ["TS/20220717/01/",
          "TS/20220717/02/",
          "TS/20220717/03/",
          "TS/20220717/04/",
          "TS/20220717/05/"
         ]



# General parameters 
filtering = False
thresold_d = 2                 # m/s
thresold_a = 1                 # deg/s
thresold_e = 1                 # deg/s
limit_time_interval = 1        # s
path_output = "./data/prediction/"
B.epsilon = 1e-8
Mode = "L"
limit_search = limit_time_interval
size_interval = 6           # Minimum time size sub-interval 
delta_t = 1                 # Value to remove points near edge of time intervals
save = True

# GP parameters
verbose=False
Number_restart = 1000
noise_GP = 0                # Noise of GP
variance_GP = 1             # Variance of GP
lengthscale_GP = 1          # Lengthscale of GP

for fname, opath in zip(file,output): 
    if(not filtering):
        path = opath + "raw/"
    else:
        path = opath + "filtered/"
        
    if(filtering):
        t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = ttf.read_rosbag_theodolite_without_tf_raw_data_pre_filtered(fname)
        index_1_f = ttf.thresold_raw_data(t1, d1, a1, e1, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
        index_2_f = ttf.thresold_raw_data(t2, d2, a2, e2, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
        index_3_f = ttf.thresold_raw_data(t3, d3, a3, e3, thresold_d, thresold_a*3.1415926/180, thresold_e*3.1415926/180, limit_time_interval)
        t1 = t1[index_1_f]
        t2 = t2[index_2_f]
        t3 = t3[index_3_f]
        tp1 = tp1[index_1_f].T
        tp2 = tp2[index_2_f].T
        tp3 = tp3[index_3_f].T
        print(len(t1),len(t2),len(t3))
    else:
        t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = ttf.read_rosbag_theodolite_without_tf_raw_data(fname)
        print(len(t1),len(t2),len(t3))

    time_origin = np.min([t1[0],t2[0],t3[0]])

    start_time = time.time()

    t1 = t1 - np.ones_like(t1)*time_origin
    t2 = t2 - np.ones_like(t2)*time_origin
    t3 = t3 - np.ones_like(t3)*time_origin

    list_interval, list_time = tfu.split_time_interval_all_data(t1, t2, t3, limit_time_interval)
    list_trajectories_split = tfu.merge_interval(list_interval, list_time, t1, t2, t3, limit_search)

    Prediction_1 = []
    Prediction_2 = []
    Prediction_3 = []
    T_prediction = []

    for i in tqdm(list_trajectories_split):

        index_1 = np.array([i[0,0],i[1,0]])
        index_2 = np.array([i[0,1],i[1,1]])
        index_3 = np.array([i[0,2],i[1,2]])

        begin = np.max([t1[index_1[0]], t2[index_2[0]], t3[index_3[0]]])+delta_t
        end = np.min([t1[index_1[1]], t2[index_2[1]], t3[index_3[1]]])-delta_t

        if(abs(end-begin)>size_interval and begin<end):

            rate = 10  #Hz
            T_prediction_init = torch.from_numpy(np.arange(begin, end, 1/rate))

            if(Mode == "MGPO" or Mode == "All"):
                T_MGPO, S_MGPO = GPf.data_training_MGPO(t1, t2, t3, tp1, tp2, tp3, index_1, index_2, index_3)
                print("Training")
                m = GPf.training_MGPO(Number_restart, verbose, T_MGPO, S_MGPO)
                print("Prediction")
                for i in T_prediction_init.numpy():
                    T_prediction.append(i+time_origin)
                    P1_MGPO, P2_MGPO, P3_MGPO = GPf.unit_prediction_MGPO(i, m)
                    Prediction_1.append(P1_MGPO)
                    Prediction_2.append(P2_MGPO)
                    Prediction_3.append(P3_MGPO)

            if(Mode == "GP" or Mode == "All"):
                T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3 = GPf.data_training_GP(t1, t2, t3, tp1, tp2, tp3, index_1, index_2, index_3)
                print("Training")
                mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3 = GPf.training_GP(Number_restart, verbose, T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3)
                print("Prediction")
                for i in T_prediction_init.numpy():
                    T_prediction.append(i+time_origin)
                    P1_GP, P2_GP, P3_GP = GPf.unit_prediction_GP(i, mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3)
                    Prediction_1.append(P1_GP)
                    Prediction_2.append(P2_GP)
                    Prediction_3.append(P3_GP)

            if(Mode == "L" or Mode == "All"):
                T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3 = GPf.data_training_L(t1, t2, t3, tp1, tp2, tp3, index_1, index_2, index_3)
                print("Prediction")
                mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3 = GPf.linear_interpolation(T1, X1, Y1, Z1, T2, X2, Y2, Z2,T3, X3, Y3, Z3)

                for i in T_prediction_init.numpy():
                    T_prediction.append(i+time_origin)
                    P1_GP, P2_GP, P3_GP = GPf.linear_prediction(i, time_origin, mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3)
                    Prediction_1.append(P1_GP)
                    Prediction_2.append(P2_GP)
                    Prediction_3.append(P3_GP)

            if(Mode=="SGP"):
                prediction_value = T_prediction_init.numpy()
                # Prepare data for training
                T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3 = GPf.data_training_GP_stheno(t1, t2, t3, tp1, tp2, tp3, index_1, index_2, index_3)
                # Training for each axis
                m_X1, v_X1 = GPf.GP_function_stheno(x=prediction_value, x_obs=T1, y_obs=X1, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                m_Y1, v_Y1 = GPf.GP_function_stheno(x=prediction_value, x_obs=T1, y_obs=Y1, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                m_Z1, v_Z1 = GPf.GP_function_stheno(x=prediction_value, x_obs=T1, y_obs=Z1, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                m_X2, v_X2 = GPf.GP_function_stheno(x=prediction_value, x_obs=T2, y_obs=X2, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                m_Y2, v_Y2 = GPf.GP_function_stheno(x=prediction_value, x_obs=T2, y_obs=Y2, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                m_Z2, v_Z2 = GPf.GP_function_stheno(x=prediction_value, x_obs=T2, y_obs=Z2, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                m_X3, v_X3 = GPf.GP_function_stheno(x=prediction_value, x_obs=T3, y_obs=X3, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                m_Y3, v_Y3 = GPf.GP_function_stheno(x=prediction_value, x_obs=T3, y_obs=Y3, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)
                m_Z3, v_Z3 = GPf.GP_function_stheno(x=prediction_value, x_obs=T3, y_obs=Z3, variance=variance_GP, lengthscale=lengthscale_GP, noise_init=noise_GP, optimization_nb=Number_restart)

                # Save prediction at time i
                for i,mx,my,mz,vx,vy,vz in zip(T_prediction_init.numpy(), m_X1, m_Y1, m_Z1, v_X1, v_Y1, v_Z1):
                    Prediction_1.append(np.array([i+time_origin, mx ,my, mz, vx, vy, vz]))
                    T_prediction.append(i+time_origin)
                for i,mx,my,mz,vx,vy,vz in zip(T_prediction_init.numpy(), m_X2, m_Y2, m_Z2, v_X2, v_Y2, v_Z2):
                    Prediction_2.append(np.array([i+time_origin, mx ,my, mz, vx, vy, vz]))
                for i,mx,my,mz,vx,vy,vz in zip(T_prediction_init.numpy(), m_X3, m_Y3, m_Z3, v_X3, v_Y3, v_Z3):
                    Prediction_3.append(np.array([i+time_origin, mx ,my, mz, vx, vy, vz]))

    stop_time = time.time()
    print(stop_time - start_time)

    print("Interpolation finished !")

    if save:
        if(Mode == "MGPO" or Mode == "All"):
            if(filtering):
                trajectoire = "f-"+str(thresold_d)+"-"+str(thresold_a)+"-"+str(thresold_e)+"-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-MGPO-"+str(Number_restart)
            else:
                trajectoire = "nf-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-MGPO-"+str(Number_restart)

            tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_1, path_output+path+trajectoire+ "_1.csv")
            tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_2, path_output+path+trajectoire+ "_2.csv")
            tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_3, path_output+path+trajectoire+ "_3.csv")
        if(Mode == "GP" or Mode == "All"):
            if(filtering):
                trajectoire = "f-"+str(thresold_d)+"-"+str(thresold_a)+"-"+str(thresold_e)+"-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-GP-"+str(Number_restart)
            else:
                trajectoire = "nf-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-GP-"+str(Number_restart)

            tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_1, path_output+path+trajectoire+ "_1.csv")
            tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_2, path_output+path+trajectoire+ "_2.csv")
            tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_3, path_output+path+trajectoire+ "_3.csv")
        if(Mode == "SGP" or Mode == "All"):
            if(filtering):
                trajectoire = "f-"+str(thresold_d)+"-"+str(thresold_a)+"-"+str(thresold_e)+"-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-SGP-"+str(Number_restart)
            else:
                trajectoire = "nf-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-SGP-"+str(Number_restart)

            tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_1, path_output+path+trajectoire+ "_1.csv")
            tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_2, path_output+path+trajectoire+ "_2.csv")
            tfu.Convert_data_prediction_to_csv(T_prediction, Prediction_3, path_output+path+trajectoire+ "_3.csv")
        if(Mode == "L" or Mode == "All"):
            if(filtering):
                trajectoire = "f-"+str(thresold_d)+"-"+str(thresold_a)+"-"+str(thresold_e)+"-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-L"
            else:
                trajectoire = "nf-"+str(limit_time_interval)+"-"+str(size_interval)+"-"+str(delta_t)+"-L"

            if save:
                tfu.Convert_datap_to_csv(T_prediction, Prediction_1, path_output+path+trajectoire+ "_1.csv")
                tfu.Convert_datap_to_csv(T_prediction, Prediction_2, path_output+path+trajectoire+ "_2.csv")
                tfu.Convert_datap_to_csv(T_prediction, Prediction_3, path_output+path+trajectoire+ "_3.csv")

    print("Saved !")

Number of data for theodolites: [517 518 517]
Bad measures: 0
495 493 496


  0%|                                                                                                                                                                                        | 0/2 [00:00<?, ?it/s]

GP(0, 434.3814697265625 * (EQ() > 0.8788583874702454))
GP(0, 4.744624137878418 * (EQ() > 0.8027766942977905))
GP(0, 0.20916277170181274 * (EQ() > 1.1378467082977295))
GP(0, 430.88397216796875 * (EQ() > 0.8799142241477966))
GP(0, 250.09422302246094 * (EQ() > 0.8803264498710632))
GP(0, 0.2653728127479553 * (EQ() > 0.9976757168769836))
GP(0, 51.51428985595703 * (EQ() > 0.8473787903785706))
GP(0, 414.2235412597656 * (EQ() > 0.8465374708175659))


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [06:38<00:00, 199.16s/it]

GP(0, 0.09957518428564072 * (EQ() > 1.0483992099761963))
398.33489060401917
Interpolation finished !
Conversion done !


...

GP(0, 0.20150145888328552 * (EQ() > 0.9772828817367554))
GP(0, 206.26016235351562 * (EQ() > 0.9118435978889465))
GP(0, 0.4115113615989685 * (EQ() > 1.1129344701766968))
GP(0, 0.26461872458457947 * (EQ() > 1.3748483657836914))
GP(0, 203.89390563964844 * (EQ() > 0.9641488790512085))
GP(0, 21.257610321044922 * (EQ() > 0.9232994914054871))
GP(0, 0.550814151763916 * (EQ() > 1.085489273071289))
GP(0, 5.0741729736328125 * (EQ() > 1.2476110458374023))
GP(0, 200.7961883544922 * (EQ() > 0.9570150971412659))


 91%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏               | 20/22 [15:08<00:50, 25.16s/it]

GP(0, 0.25851285457611084 * (EQ() > 0.9941381812095642))
GP(0, 98.20864868164062 * (EQ() > 0.9172630310058594))
GP(0, 0.09684979170560837 * (EQ() > 1.0834286212921143))
GP(0, 0.24565288424491882 * (EQ() > 1.8194599151611328))
GP(0, 99.5776596069336 * (EQ() > 0.8969355821609497))
GP(0, 6.788206577301025 * (EQ() > 1.2395904064178467))
GP(0, 0.5011767148971558 * (EQ() > 0.8303961157798767))
GP(0, 2.6242477893829346 * (EQ() > 1.0005055665969849))
GP(0, 99.7082748413086 * (EQ() > 0.9609231352806091))


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [15:33<00:00, 42.44s/it]

GP(0, 0.23021796345710754 * (EQ() > 0.8323732018470764))
933.7930750846863
Interpolation finished !
Conversion done !
Conversion done !
Conversion done !
Saved !
